<a href="https://colab.research.google.com/github/kumardevanshuu/automated_data_analysis_using_langchain/blob/main/Automated_Data_Analysis_Using_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain-groq tabulate openpyxl streamlit langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00


In [2]:
import pandas as pd

from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain import LLMChain
from langchain.chains import LLMChain

import os
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_groq import ChatGroq

In [3]:
llm = ChatGroq(temperature=0.8, groq_api_key='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', model_name="llama3-70b-8192")

In [4]:
prompt_template = """
I have the following CSV data with the columns: "{columns}".
Data is in a dataframe called 'df' already.
Don't give any description or explanation or any English sentence,
just write relevant Python code and store output in a variable called result.
Please generate a Python script using this 'df' as input dataframe and pandas to answer this question: "{question}".
Do not write any Python script which alters the dataframe 'df'.
Write only read-only Python script.
"""

In [5]:
template = PromptTemplate(
    input_variables=["columns", "question"],
    template=prompt_template,
)


In [6]:
llm_chain = LLMChain(prompt=template, llm=llm)

<ipython-input-6-7b6445c82a54>:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=template, llm=llm)


In [7]:
uploaded_file = "/content/customers-100.csv" #### use the file from your HTML Page here

In [9]:
df = pd.DataFrame()
if uploaded_file.endswith('.csv'):
    df = pd.read_csv(uploaded_file)
elif uploaded_file.endswith(('.xls', '.xlsx')): #### add a condition for google sheet
    df = pd.read_excel(uploaded_file)

In [15]:
question = "What is the customer ID of Darren " # Question here from question box

In [16]:
#This is your submit button code

if question:
    try:
        # Generate Python code to answer the question
        python_script = llm_chain.invoke({
            "columns": ", ".join(df.columns),
            "question": question
        })

        # Display the generated Python code
        print("### Generated Python Code:")
        print(python_script['text'].strip('`'))

        # Execute the generated Python code
        exec_globals = {"df": df, "pd": pd}
        exec_locals = {}
        python_script['text'] = python_script['text'].strip('`')
        exec(python_script['text'], exec_globals, exec_locals)

        # If a result variable is present, store and print it
        answer = exec_globals.get('result', exec_locals.get('result'))
        if answer is not None:
            print("### Result:")
            print(answer)
        else:
            print(" The code did not produce a 'result' variable.")
    except Exception as e:
        print(f"Error running the code: {e}")

### Generated Python Code:

result = df.loc[df['First Name'] == 'Darren', 'Customer Id'].values

### Result:
['EA4d384DfDbBf77']
